In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

# Test runs of the SEIR model for Belgium - coronavirus

**Author:** Pierre de Buyl  
**Date:** 11 march 2020  
**Licence:** [CC-BY](https://creativecommons.org/licenses/by/4.0/)

*Warning:* I am not an epidemiologist. I wrote this notebook to have a starting
point for comparing estimates that I found online, for instance the one by Nicolas
Vandewalle show below.

I used a standard epidemiological model (SEIR) along with parameters from a recent
article by a team of chinese scientists for the case of Wuhan.

The absolute values are likely incorrect. The trends are good to know.

![](figure_vdwnico_20200308.jpeg)

Source: [@vdwnico](https://twitter.com/vdwnico/status/1236542044685959168) on Twitter, 8 march 2020.

## Data from Belgian government

Below, I write down the number of new cases in Belgium since march 2.

`data_I` is the total of infected cases (there are no recoveries recorded yet).

In [ ]:
new_I = [2, 6, 5, 10, 27, 59, 60, 31, 39, 28, 47] # starting march 1
data_I = np.cumsum(new_I)
data_dates = np.arange('2020-03-01', len(new_I), dtype=np.datetime64)

## Definition of the model and parameters

I take the SEIR model defined in Wang et al. Cell Discovery (2020) 6:10 https://doi.org/10.1038/s41421-020-0148-0



In [ ]:
# parameters for the model, page 6

σ = 1/5.2 # inverse of mean period
γ = 1/18  # time for which a case should be recovered
#R0 = 3.1  # estimates between 1.9 and 3.1

#β = R0*γ

N = 11e6  # population size

# definition of model page 6
def SEIR(x, t, R0):
    S, E, I, R = x
    β = R0*γ
    return [
            - β*I*S/N,
            β*I*S/N - σ*E,
            σ*E - γ*I,
            γ*I,
           ]

## Estimate for Belgium

I start with the record of 2 infected cases on march 2 at https://www.info-coronavirus.be/fr/2020/03/01/deuxieme-infection-au-covid-19-en-belgique/ and list the new cases day per day (see above).

The "day 0" is thus march 2 and the initial number of infected people is 2.

For estimating the healthcare network saturation, I use the estimate of Philippe Devos regarding the
availability of intensive care beds and the estimate of 6.1% of infected people needing intensive care
(no other information is taken from that article).

Article by Philippe Devos in La Libre Belgique, 2 march 2020:
https://www.lalibre.be/debats/opinions/coronavirus-sans-mesures-de-precaution-drastiques-on-risque-d-avoir-850-000-personnes-infectees-et-50-000-morts-en-belgique-5e5cf60f9978e23106a0bfd9

To circumvent the low statistics of the first few days, I allow to delay the initial condition
of the SEIR model by an offset of `start_day`, taking the corresponding cumulated number of cases
for I0.


In [ ]:
start_day = 3
I0 = data_I[start_day]
E0 = 20*I0 # number of initially exposed people, see Wang et al

t = np.arange(0, 200, dtype=float) # running for 100 days
t_display = np.arange('2020-03-02', len(t), dtype=np.datetime64) + start_day


In [ ]:
plt.figure()
plt.plot(data_dates, data_I, marker='o', label='reported I')

R0 = 1.9
model = odeint(SEIR, [N-I0-E0, E0, I0, 0], t, args=(R0,))
plt.plot(t_display, model[:,2], label=f'model I. R0={R0}')

R0 = 3.1
model = odeint(SEIR, [N-I0-E0, E0, I0, 0], t, args=(R0,))
plt.plot(t_display, model[:,2], label=f'model I. R0={R0}')

plt.yscale('log')

plt.xlabel('time')

available_icu_beds = 140
fraction_of_icu_need = 0.061
saturation_of_icu_beds = 140/fraction_of_icu_need
plt.axhline(saturation_of_icu_beds, label='saturation', c='k', ls='--')

ylo, yhi = plt.ylim()
plt.ylim(data_I[0], yhi)

plt.legend()
plt.grid()
#plt.savefig('seir_model_start_2.png')

## Risk of under-reporting

Again, as a test, I use below twice the number of reported cases to
understand the risk related to under-reporting.

If the actual number of cases is twice as high, the peak infection arises
earlier (mid-end july instead of mid-august) and the saturation of ICU beds
occurs late march instead of mid-april.

In [ ]:
I0 = 2*I0
E0 = 20*I0

plt.figure()
plt.plot(data_dates, 2*data_I, marker='o', label='reported I')

R0 = 1.9
model = odeint(SEIR, [N-I0-E0, E0, I0, 0], t, args=(R0,))
plt.plot(t_display, model[:,2], label=f'model I. R0={R0}')

R0 = 3.1
model = odeint(SEIR, [N-I0-E0, E0, I0, 0], t, args=(R0,))
plt.plot(t_display, model[:,2], label=f'model I. R0={R0}')

plt.yscale('log')

plt.xlabel('time')

available_icu_beds = 140
fraction_of_icu_need = 0.061
saturation_of_icu_beds = 140/fraction_of_icu_need
plt.axhline(saturation_of_icu_beds, label='saturation', c='k', ls='--')

ylo, yhi = plt.ylim()
plt.ylim(data_I[0], yhi)

plt.legend()
plt.grid()
#plt.savefig('seir_model_start_2.png')

## Observations

The initial data corresponds vaguely with the model. It does not stand
closer inspection of the figure by zooming.

With respect to Nicolas Vandewalle, I find a later peak in august 2020
instead of end may.

For comparison, I also added the healthcare network treshold suggested
by Philippe Devos in La Libre Belgique. This treshold, should the SEIR model
be realistic and the estimate for the number of ICU beds hold, would be reached
around the 10 april for $R_0=3.1$ and mid-may for $R_0=1.9$.

To assess the risk of under-reporting, I used the double amount of infected cases.